# Descripción del proyecto

Trabajas en la compañía de extracción de petróleo OilyGiant. Tu tarea es encontrar los mejores lugares donde abrir 200 pozos nuevos de petróleo.

Para completar esta tarea, tendrás que realizar los siguientes pasos:

* Leer los archivos con los parámetros recogidos de pozos petrolíferos en la región seleccionada: calidad de crudo y volumen de reservas.
* Crear un modelo para predecir el volumen de reservas en pozos nuevos.
* Elegir los pozos petrolíferos que tienen los valores estimados más altos.
* Elegir la región con el beneficio total más alto para los pozos petrolíferos seleccionados.

Tienes datos sobre muestras de crudo de tres regiones. Ya se conocen los parámetros de cada pozo petrolero de la región. Crea un modelo que ayude a elegir la región con el mayor margen de beneficio. Analiza los beneficios y riesgos potenciales utilizando la técnica bootstrapping.


**CONDICIONES**

* Solo se debe usar la regresión lineal para el entrenamiento del modelo.
* Al explorar la región, se lleva a cabo un estudio de 500 puntos con la selección de los mejores 200 puntos para el cálculo del beneficio.
* El presupuesto para el desarrollo de 200 pozos petroleros es de 100 millones de dólares.
* Un barril de materias primas genera 4.5 USD de ingresos. El ingreso de una unidad de producto es de 4500 dólares (el volumen de reservas está expresado en miles de barriles).
* Después de la evaluación de riesgo, mantén solo las regiones con riesgo de pérdidas inferior al 2.5%. De las que se ajustan a los criterios, se debe seleccionar la región con el beneficio promedio más alto.

Los datos son sintéticos: los detalles del contrato y las características del pozo no se publican.

## Descripción de datos

Los datos de exploración geológica de las tres regiones se almacenan en archivos:

* `/datasets/geo_data_0.csv`
* `/datasets/geo_data_1.csv`
* `/datasets/geo_data_2.csv`
  
**Características**
* `id` — identificador único de pozo de petróleo
* `f0`, `f1`, `f2` — tres características de los puntos (su significado específico no es importante, pero las características en sí son significativas)

**Objetivo**
* `product` — volumen de reservas en el pozo de petróleo (miles de barriles).


## Inicialización

In [26]:
# Cargar todas las librerías
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split #
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler #
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle

## Importar archivos

### Geo Data 0

In [27]:
geo_0 = pd.read_csv('/datasets/geo_data_0.csv')

print(f'Duplicados : {geo_0.duplicated().sum()}\n')
geo_0.info()
geo_0

Duplicados : 0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
...,...,...,...,...,...
99995,DLsed,0.971957,0.370953,6.075346,110.744026
99996,QKivN,1.392429,-0.382606,1.273912,122.346843
99997,3rnvd,1.029585,0.018787,-1.348308,64.375443
99998,7kl59,0.998163,-0.528582,1.583869,74.040764


### Geo Data 1

In [28]:
geo_1 = pd.read_csv('/datasets/geo_data_1.csv')

print(f'Duplicados : {geo_1.duplicated().sum()}\n')
geo_1.info()
geo_1

Duplicados : 0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
...,...,...,...,...,...
99995,QywKC,9.535637,-6.878139,1.998296,53.906522
99996,ptvty,-10.160631,-12.558096,5.005581,137.945408
99997,09gWa,-7.378891,-3.084104,4.998651,137.945408
99998,rqwUm,0.665714,-6.152593,1.000146,30.132364


### Geo Data 2

In [29]:
geo_2 = pd.read_csv('/datasets/geo_data_2.csv')

print(f'Duplicados : {geo_2.duplicated().sum()}\n')
geo_2.info()
geo_2

Duplicados : 0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
...,...,...,...,...,...
99995,4GxBu,-1.777037,1.125220,6.263374,172.327046
99996,YKFjq,-1.261523,-0.894828,2.524545,138.748846
99997,tKPY3,-1.199934,-2.957637,5.219411,157.080080
99998,nmxp2,-2.419896,2.417221,-5.548444,51.795253


### Preparar los datos

<div class="alert alert-block" style="background-color: #eceeef; color: black;">
Se puede observar que no se presentan duplicados, el tipo de datos son correctos y no se presentan valores ausentes. Por lo tanto se puede continuar con el procedimiento. 
    <a class=“tocSkip”></a>
</div>

## Entrenamiento y evaluación del modelo `GEO DATA 0`

### Segmentación de datos

tomando como **características**
* `id` — identificador único de pozo de petróleo
* `f0`, `f1`, `f2` — tres características de los puntos (su significado específico no es importante, pero las características en sí son significativas)

y como **objetivo**
* `product` — volumen de reservas en el pozo de petróleo (miles de barriles).

In [30]:
features = geo_0.drop(['id', 'product'], axis=1)
target = geo_0['product']

<div class="alert alert-block" style="background-color: #eceeef; color: black;">
Se segmentan los datos fuente en un 75% para el entrenamiento, pues se necesita tener suficiente información para que el modelo aprenda. Un 25% para la validación, se necesita tener una muestra representativa para evaluar durante el desarrollo.
    <a class=“tocSkip”></a>
</div> 

In [31]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345)

In [32]:
splits = {'Entrenamiento': features_train,
          'Validación': features_valid}

total = geo_0.shape[0]

for nombre, conjunto in splits.items():
    porcentaje = 100 * len(conjunto) / total
    print(f'Porcentaje del conjunto de {nombre.lower()}: {porcentaje:.1f} %')

Porcentaje del conjunto de entrenamiento: 75.0 %
Porcentaje del conjunto de validación: 25.0 %


### Escalado de características

In [33]:
numeric = ['f0', 'f1', 'f2']

features_train = features_train.copy()
features_valid = features_valid.copy()

scaler = StandardScaler()

features_train[numeric] = scaler.fit_transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])

features_train.head()

,f0,f1,f2
27212,-0.544828,1.390264,-0.094959
7866,1.455912,-0.480422,1.209567
62041,0.260460,0.825069,-0.204865
70185,-1.837105,0.010321,-0.147634
82230,-1.299243,0.987558,1.273181


### Entrenamiento del modelo

In [34]:
model = LinearRegression()

model.fit(features_train, target_train)
predictions = model.predict(features_valid)

In [35]:
rmse = mean_squared_error(target_valid, predictions, squared=False)
print(f'RMSE para el modelo de Regresión Lineal en el conjunto de validación: {rmse:.4f}\n')
print(f'El volumen medio de reservas predicho: {predictions.mean():.4f}')

RMSE para el modelo de Regresión Lineal en el conjunto de validación: 37.5794

El volumen medio de reservas predicho: 92.5926


## Entrenamiento y evaluación de los modelos

In [36]:
def reserves(geo_data, test_size=0.25, random_state=12345):
    # Segmentación de datos
    features = geo_data.drop(['id', 'product'], axis=1)
    target = geo_data['product']
    features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=test_size, random_state=random_state)

    # Escalado de características
    numeric = ['f0', 'f1', 'f2']
    features_train = features_train.copy()
    features_valid = features_valid.copy()
    scaler = StandardScaler()
    features_train[numeric] = scaler.fit_transform(features_train[numeric])
    features_valid[numeric] = scaler.transform(features_valid[numeric])

    # Entrenamiento del modelo
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions = pd.Series(model.predict(features_valid))
    rmse = mean_squared_error(target_valid, predictions, squared=False)
    volume = predictions.mean()
    target_valid = target_valid.reset_index(drop=True)
    
    return target_valid, predictions, rmse, volume

### Geo Data 0

In [37]:
target_valid_0, predictions_0, rmse_0, volume_0 = reserves(geo_0)

print(f'RMSE para el modelo de Regresión Lineal en el conjunto de validación: {rmse_0:.4f}\n')
print(f'El volumen medio de reservas predicho: {predictions_0.mean():.4f}')

RMSE para el modelo de Regresión Lineal en el conjunto de validación: 37.5794

El volumen medio de reservas predicho: 92.5926


### Geo Data 1

In [38]:
target_valid_1, predictions_1, rmse_1, volume_1 = reserves(geo_1)

print(f'RMSE para el modelo de Regresión Lineal en el conjunto de validación: {rmse_1:.4f}\n')
print(f'El volumen medio de reservas predicho: {predictions_1.mean():.4f}')

RMSE para el modelo de Regresión Lineal en el conjunto de validación: 0.8931

El volumen medio de reservas predicho: 68.7285


### Geo Data 2

In [39]:
target_valid_2, predictions_2, rmse_2, volume_2 = reserves(geo_2)

print(f'RMSE para el modelo de Regresión Lineal en el conjunto de validación: {rmse_2:.4f}\n')
print(f'El volumen medio de reservas predicho: {predictions_2.mean():.4f}')

RMSE para el modelo de Regresión Lineal en el conjunto de validación: 40.0297

El volumen medio de reservas predicho: 94.9650


<div class="alert alert-block" style="background-color: #eceeef; color: black;">
    
La región 1 tiele el modelo más preciso, pero su volumen predicho está muy por debajo, aproximadamente a la mitad (68.72) del umbral rentable (111.1).

La región 0 y la región 2, tiene modelos menos precisos con valores similares de rsme (37.58 y 40.03 respectivamente), pero predicen el mayor volumen. En este caso, lo más importante es el volumen esperado, por lo que se selecciona la región 2, ya que es la que cuenta con el mayor volumen (94.96).
    <a class=“tocSkip”></a>
</div>

## Cálculo de ganancias

<div class="alert alert-block" style="background-color: #eceeef; color: black;">
    
Dada la inversión de 100 millones por 200 pozos petrolíferos, de media un pozo petrolífero debe producir al menos un valor de 500,000 dólares en unidades para evitar pérdidas, esto es equivalente a <b>111.1 unidades</b>.

La región 2 tiene el mayor volumen medio de reservas predicho de las 3 regiones, aunque sigue siendo inferior al esperado para evitar pérdidas. Aparentemente, es la región recomendada para el desarrollo de los pozos petroleros.

Con el fin de dar un mejor resultado, se seleccionarán los mejores 200 pozos petroleros de cada región y se calcularán las ganancias y beneficios.
    <a class=“tocSkip”></a>
</div>

In [40]:
investment = 100000000
wells = 200
income_per_unit = 4500
min_income_per_well = investment / wells
min_units_per_well = min_income_per_well/income_per_unit

print(f'Cantidad mínima requerida por pozo: {min_units_per_well:.1f}')

Cantidad mínima requerida por pozo: 111.1


In [41]:
def profit(target_valid, predictions):
    predictions = predictions.reset_index(drop=True)
    target_valid = target_valid.reset_index(drop=True)
    
    top_positions = predictions.sort_values(ascending=False).head(wells).index
    target = target_valid.iloc[top_positions]
    
    volume = target.sum()
    income = target * income_per_unit

    best = pd.DataFrame({'product': target, 'income': income})
    total_income = income.sum() - investment
    
    return best, volume, total_income

In [42]:
best_0, volume_0, total_income_0 = profit(target_valid_0, predictions_0)
best_1, volume_1, total_income_1 = profit(target_valid_1, predictions_1)
best_2, volume_2, total_income_2 = profit(target_valid_2, predictions_2)

In [43]:
def results(regions_data):
    best_region = max(enumerate(regions_data), key=lambda x: x[1][1])
    
    for i, (volume, total_income) in enumerate(regions_data):
        print(f'  Región {i} - Ganancia potencial de los {wells} pozos principales: $ {total_income:,.2f}')
        print(f'  Región {i} - Volumen objetivo de reserva es de {volume.mean():,.2f} miles de barriles')
        print('-' * 80, '\n')

    i_best, (_,max_income) = best_region
    print(f'  La Región {i_best} es la más rentable con un ingreso de $ {max_income:,.2f}\n')

In [44]:
regions_data =[
    (volume_0, total_income_0),
    (volume_1, total_income_1),
    (volume_2, total_income_2)]

results = results(regions_data)

  Región 0 - Ganancia potencial de los 200 pozos principales: $ 33,208,260.43
  Región 0 - Volumen objetivo de reserva es de 29,601.84 miles de barriles
-------------------------------------------------------------------------------- 

  Región 1 - Ganancia potencial de los 200 pozos principales: $ 24,150,866.97
  Región 1 - Volumen objetivo de reserva es de 27,589.08 miles de barriles
-------------------------------------------------------------------------------- 

  Región 2 - Ganancia potencial de los 200 pozos principales: $ 27,103,499.64
  Región 2 - Volumen objetivo de reserva es de 28,245.22 miles de barriles
-------------------------------------------------------------------------------- 

  La Región 0 es la más rentable con un ingreso de $ 33,208,260.43



<div class="alert alert-block" style="background-color: #eceeef; color: black;">
En cuestión de ganancia potencial, la región 0 es la mejor para el desarrollo de pozos.
    <a class=“tocSkip”></a>
</div>

## Cálculo de riesgos

<div class="alert alert-block" style="background-color: #eceeef; color: black;">
Se empleará la técnica de <b>bootstrapping</b> con <b>1,000</b> muestras, con un intervalo de confianza de <b>95 %</b>. Esto con el fin de obtener los riesgos y ganancias para cada región.
    <a class=“tocSkip”></a>
</div>

In [45]:
def profit_bootstrapping(predictions, target_valid, interval, bootstrap_samples=1000, points=500):
    
    profits = []
    state = np.random.RandomState(12345)
    
    for _ in range(bootstrap_samples):
        sample_index = state.choice(len(target_valid), size=points, replace=True)
        sample_target = target_valid.iloc[sample_index]
        sample_predictions = predictions.iloc[sample_index]
        
        _, _, profit_value = profit(sample_target, sample_predictions)
        profits.append(profit_value)

    profits = pd.Series(profits)
    mean_profits = profits.mean()
    
    alpha = (100 - interval) / 2
    confidence_interval = np.percentile(profits, [alpha, 100 - alpha])
    risk_of_lose = (profits < 0).mean() * 100
    
    return profits, mean_profits, confidence_interval, risk_of_lose

In [54]:
def losses_gains(ci):
    if ci < 0:
        ci_text = f"una pérdida de $ {abs(ci):,.2f} millones"
    else:
        ci_text = f"una ganancia de $ {abs(ci):,.2f} millones"
    return ci_text
    

def print_results(regions_data):

    for i, (mean, ci, risk) in enumerate(regions_data):
        print(f' Región {i}')
        print(f' Promedio de ganancias: ${mean:,.2f}')
        print(f' Con un 95% de confianza, la ganancia real estará entre {losses_gains(ci[0])} y {losses_gains(ci[1])}.')
        print(f' Riesgo de pérdidas: {risk:.1f}%')
        print('-' * 125, '\n')
    
    filtered = [(i, data) for i, data in enumerate(regions_data) if data[2]<2.5]

    if not filtered:
        print('Ninguna región cumple con el criterio de un riesgo menor a 2.5%')
    else:
        best_region = max(filtered, key=lambda x: x[1][0])
        i_best, (max_mean,_,risk_min) = best_region
        print(f'  La Región {i_best} es la más rentable con un riesgo de {risk_min} %, con un ingreso promedio de $ {max_mean:,.2f}\n')

In [55]:
condifence_interval = 95

profits_0, mean_0, ci_0, risk_0 = profit_bootstrapping(predictions_0, target_valid_0, condifence_interval)
profits_1, mean_1, ci_1, risk_1 = profit_bootstrapping(predictions_1, target_valid_1, condifence_interval)
profits_2, mean_2, ci_2, risk_2 = profit_bootstrapping(predictions_2, target_valid_2, condifence_interval)

regions_data =[
    (mean_0, ci_0, risk_0),
    (mean_1, ci_1, risk_1),
    (mean_2, ci_2, risk_2)]

results = print_results(regions_data)

 Región 0
 Promedio de ganancias: $3,961,649.85
 Con un 95% de confianza, la ganancia real estará entre una pérdida de $ 1,112,155.46 millones y una ganancia de $ 9,097,669.42 millones.
 Riesgo de pérdidas: 6.9%
----------------------------------------------------------------------------------------------------------------------------- 

 Región 1
 Promedio de ganancias: $4,560,451.06
 Con un 95% de confianza, la ganancia real estará entre una ganancia de $ 338,205.09 millones y una ganancia de $ 8,522,894.54 millones.
 Riesgo de pérdidas: 1.5%
----------------------------------------------------------------------------------------------------------------------------- 

 Región 2
 Promedio de ganancias: $4,044,038.67
 Con un 95% de confianza, la ganancia real estará entre una pérdida de $ 1,633,504.13 millones y una ganancia de $ 9,503,595.75 millones.
 Riesgo de pérdidas: 7.6%
-------------------------------------------------------------------------------------------------------------

<div class="alert alert-block" style="background-color: #eceeef; color: black;">

<b>Observaciones</b>
    
Al analizar únicamente el volumen medio de extracción predicho, la región 2 destaca como la mejor opción, al ser la que más se aproxima al volumen esperado, lo cual sugiere un buen desempeño en términos de cantidad de producto extraído.

Sin embargo, al evaluar la ganancia neta estimada, la región más rentable es la región 0, con una ganancia potencial de \$ 33,208,260.43, superior a la obtenida en la región 1 (\$ 24,150,866.97) y a la región 2 (\$ 27,103,499.64).

No obstante, al aplicar la técnica bootstrapping para evaluar el riesgo asociado a las predicciones, se puede observar que tanto en la región 0 como en la región 2, el límite inferior del intervalo de confianza es negativo. Esto indica que existe una probabilidad real de <i>pérdidas</i> económicas. En efecto, el riesgo de pérdidas de la región 0 es de 6.9 %, y en la región 2, de 7.6%, ambos superiores al umbral aceptable del 2.5 % establecido en el proyecto.

Por otro lado, la región 1, ofrece un equilibrio óptimo entre rentabilidad y riesgo. Presenta una ganancia media mayor y un riesgo de pérdidas de 1.5%, el cual se encuentra por debajo del límite permitido. 

    
<b>Conclusión</b>

Al analizar detalladamente los datos mediante la técnica de bootstrapping, la cual asegura que se tomen muestras más representativas y estables. Se puede determinar que la mejor opción para el desarrollo de los 200 pozos es la <b>Región 1</b>, debido a la ganancia media mayor de \$ 4,560,451.06 y al bajo riesgo de pérdidas de 1.5 \%. Con un intervalo de confianza de 95 \% la ganancia real estará entre una ganancia de \$ 338,205.09 y \$ 8,522,894.54. Convirtiéndola en la opción más segura y recomendable para la inversión bajo las condiciones establecidas.
    <a class=“tocSkip”></a>
</div>